In [1]:
import numpy as np

# Schwefel function
$$ f(x) = 418.9829n - \sum_{i=1}^{n}x_i \sin(\sqrt{|x_i|}) $$

In [2]:
def schwefel_function(x):
    n = len(x)
    a = 418.9829
    sum_term = np.sum(x * np.sin(np.sqrt(np.abs(x))))
    result = a * n - sum_term
    return result

In [3]:
dimension = 5
range_x = [-500, 500]
random_vector = np.random.uniform(range_x, dimension)
random_vector = np.array([420.9687,420.9687,420.9687])
result = schwefel_function(random_vector)
print(f"Schwefel function value for {random_vector}: {result}")

Schwefel function value for [420.9687 420.9687 420.9687]: 3.818351251538843e-05


# Ackley function
$$ f(x) = -20 \exp\left(-0.2 \sqrt{\frac{1}{n} \sum_{i=1}^{n}x_i^2}\right) - \exp\left(\frac{1}{n}\sum_{i=1}^{n}\cos(2\pi x_i)\right) + 20 + e $$


In [4]:
def ackley_function(x):
    n = len(x)
    a = 20
    b = 0.2
    c = 2 * np.pi
    term1 = -a * np.exp(-b * np.sqrt((1/n) * np.sum(x**2)))
    term2 = -np.exp((1/n) * np.sum(np.cos(c * x)))
    result = term1 + term2 + 20 + np.exp(1)
    return result

In [5]:
dimension = 5
range_x = [-32.768, 32.768]
# random_vector = np.random.uniform(range_x, dimension)
random_vector = np.array([0,0])
result = ackley_function(random_vector)
print(f"Ackley function value for {random_vector}: {result}")

Ackley function value for [0 0]: 4.440892098500626e-16


# Evolutionary Startegy

Ackley Function: **flat and smooth fitness landscape, unimodal landscape**

Schwefel Function: **rugged landscapes, multimodal**


In [6]:
def population_initialization(population_size, chromosome_length, range_x, initial_sigma):
    min_value, max_value = range_x
    object_parameters = np.random.uniform(min_value, max_value, size=(population_size, chromosome_length))
    population = [list(object_parameter) + chromosome_length * [initial_sigma] for i, object_parameter in enumerate(object_parameters)]
    return population

In [7]:
# def parent_selection(population, num_parents):
    

Blend crossover

simulated binary crossover

In [8]:
# Randomly choose one parent for each gene 
def global_discrete_recombination(parents):
    num_genes = len(parents[0])
    offspring = np.array([np.random.choice(parents[gene]) for gene in range(num_genes)])
    return offspring

In [9]:
# Randomly select genes from either parent
def local_discrete_recombination(parents):
    offspring = np.zeros_like(parents[0])
    for i in range(len(offspring)):
        selected_parent_indices = np.random.choice(len(parents), size=1)[0]
        offspring[i] = parents[selected_parent_indices][i]
    return offspring

In [10]:
# Taking the average
def local_intermediary_recombination(parent1, parent2):
    offspring = (parent1 + parent2) / 2.0 
    return offspring

In [11]:
# Take the average of Randomly select parents for each gene
def global_intermediary_recombination(parents):
    num_genes = len(parents[0])
    offspring = np.mean(parents, axis=1)
    return offspring

$$ \sigma'_i = \sigma_i \cdot \exp\left(\tau' \cdot \mathcal{N}(0,1) + \tau \cdot \mathcal{N_i}(0,1)\right) $$

$$ x'_i = x_i + \sigma'_i \cdot \mathcal{N}(0,1) $$

$ \tau $ is proportional to $ \frac{1}{\sqrt{2n}} $

$ \tau' $ is proportional to $ \frac{1}{\sqrt{2\sqrt{n}}} $

$ \mathcal{N}(0,1) $ represents a random value drawn from a standard normal distribution,

$ \sigma'_i < \epsilon_0 $ implies $ \sigma'_i = \epsilon_0 $.



In [12]:
def uncorrelated_mutation_n_sigma(chromosome, n_sigma, epsilon=1e-10):
    random_values = np.random.randn(len(chromosome))
    tau = 1 / np.sqrt(2 * len(chromosome))
    tau_prime = 1 / np.sqrt(2 * np.sqrt(len(chromosome)))
    new_n_sigma = np.array(n_sigma) * np.exp(tau_prime * np.random.randn() + tau * random_values)
    mask = new_n_sigma < epsilon
    n_sigma[mask] = new_n_sigma[mask]
    random_values = np.random.randn(len(chromosome))
    mutated_chromosome = chromosome + n_sigma * random_values
    return mutated_chromosome, n_sigma

$$ \sigma' = \sigma \cdot \exp\left(\tau \cdot \mathcal{N}(0,1)\right) $$

$$ x'_i = x_i + \sigma' \cdot \mathcal{N}(0,1) $$

$ \tau $ is proportional to $ \frac{1}{\sqrt{n}} $

$ \mathcal{N}(0,1) $ represents a random value drawn from a standard normal distribution,

$ \sigma' < \epsilon_0 $ implies $ \sigma' = \epsilon_0 $.



In [13]:
def uncorrelated_mutation_one_sigma(chromosome, one_sigma, epsilon=1e-10):
    random_value = np.random.randn()
    tau = 1 / np.sqrt(len(chromosome))
    new_one_sigma = one_sigma
    new_one_sigma *= np.exp(tau * random_value)
    if new_one_sigma < epsilon:
        new_one_sigma = one_sigma
    random_value = np.random.randn()
    mutated_chromosome = chromosome + new_one_sigma * random_value
    return mutated_chromosome, one_sigma

In [14]:
def elitism_survival_selection(objective_function,chromosome_length, parents, offsprings):
    combined_population = np.vstack((parents, offsprings))
    chromosomes = np.array(combined_population)[:, range(0, chromosome_length)]
    if objective_function == "schwefel_function":
        new_generation_indices = np.argsort(schwefel_function(combined_population))[:len(parents)]
    elif objective_function == "ackley_function":
        new_generation_indices = np.argsort(ackley_function(combined_population))[:len(parents)]
    new_generation = combined_population[new_generation_indices]
    return new_generation

In [15]:
def genrational_survival_selection(objective_function,chromosome_length, parents, offsprings):
    chromosomes = np.array(offsprings)[:, range(0, chromosome_length)]
    new_generation = []
    if objective_function == "schwefel_function":
        new_generation = np.sort(schwefel_function(offsprings))[:len(parents)]
    elif objective_function == "ackley_function":
        new_generation = np.sort(ackley_function(offsprings))[:len(parents)]
    return new_generation


In [16]:
def calculate_fitness(objective_function, chromosome):
    fitness = objective_function(chromosome)
    return fitness

In [17]:
def generate_offspring(objective_function, population, chromosome_length, num_offspring, num_parents_involved=2):
    offsprings = []
    while len(offsprings) < num_offspring:
        if objective_function == "schwefel_function":
            selected_parents_indices = np.random.choice(len(population), size=num_parents_involved, replace=True)
            selected_parents = [population[i] for i in selected_parents_indices]
            offspring = local_discrete_recombination(selected_parents)
            offspring = uncorrelated_mutation_n_sigma(offspring[:chromosome_length], offspring[chromosome_length:])
        elif objective_function == "ackley_function":
            selected_parents_indices = np.random.choice(len(population), size=num_parents_involved, replace=True)
            selected_parents = [population[i] for i in selected_parents_indices]
            offspring = local_discrete_recombination(selected_parents)
            object_p, strategy_p = uncorrelated_mutation_n_sigma(offspring[:chromosome_length], offspring[chromosome_length:])
            offspring = list(object_p) + list(strategy_p)
        offsprings.append(offspring)
    return offsprings[:num_offspring]

In [18]:
def check_solution(objective_function, chromosome_length, population):
        chromosomes = np.array(population)[:, range(0, chromosome_length)]
        result_list = []
        if objective_function == "schwefel_function":
            result_list = [schwefel_function(x) for x in chromosomes]
            sorted_result = sorted(result_list)
        elif objective_function == "ackley_function":
            result_list = [ackley_function(x) for x in chromosomes]
            sorted_result = sorted(result_list)
        return result_list[0]

In [19]:
def evolutionary_strategy(objective_function, survival_method, max_generations, population_size, chromosome_length, num_offspring):
    offsprings = []
    population = []
    new_generation = []
    count_generations = 0
    initial_sigma = 0.1
    if objective_function == "schwefel_function":
        population = population_initialization(population_size, chromosome_length, (-500, 500), initial_sigma)
    elif objective_function == "ackley_function":
        population = population_initialization(population_size, chromosome_length, (-32.768, 32.768) ,initial_sigma)
        
    while count_generations < max_generations: # fix termination_condition
        if objective_function == "schwefel_function":
            offsprings = generate_offspring(objective_function, population, chromosome_length, num_offspring)
        elif objective_function == "ackley_function":
            offsprings = generate_offspring(objective_function, population, chromosome_length, num_offspring)
        if survival_method == "elitism":
            new_generation = elitism_survival_selection(objective_function, chromosome_length, population, offsprings)
        elif survival_method == "generational":
            new_generation = genrational_survival_selection(objective_function, chromosome_length, population, offsprings)
        population = new_generation
        count_generations += 1
    solution = check_solution(objective_function, chromosome_length, population)
    return solution
    

In [23]:
evolutionary_strategy("schwefel_function", "elitism", 200, 15, 5, 7*15)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 3 dimension(s)

In [ ]:
# Example usage:
es = EvolutionaryStrategy("schwefel_function", "elitism", 200, 15, 5, 7*15)
result = es.run()
print(result)